In [1]:
import requests
import json
import pandas as pd

from itertools import islice

# Google API Key
from config import gkey

In [2]:
chicago_df = pd.read_csv("updated_inspection.csv")
chicago_df.head()

,Unnamed: 0,Inspection ID,DBA Name,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Results,Latitude,Longitude
0,0,2268564,PIZZERIA AROMA,Restaurant,Risk 1 (High),5350 N BROADWAY,CHICAGO,IL,60640.0,2019-02-26T00:00:00,Fail,41.979651,-87.660106
1,1,2268560,PIZZERIA AROMA,Restaurant,Risk 1 (High),5350 N BROADWAY,CHICAGO,IL,60640.0,2019-02-26T00:00:00,Fail,41.979651,-87.660106
2,3,2268527,CAFECITO,Restaurant,Risk 1 (High),401-403 W FULLERTON PKWY,CHICAGO,IL,60614.0,2019-02-26T00:00:00,Pass w/ Conditions,41.925501,-87.639179
3,4,2268526,JOHNSON'S REAL ICE CREAM,Restaurant,Risk 2 (Medium),2951 N BROADWAY,CHICAGO,IL,60657.0,2019-02-26T00:00:00,Pass w/ Conditions,41.936415,-87.644128
4,5,2268524,TONY'S KITCHEN #2 INC.,Restaurant,Risk 1 (High),6325 S CENTRAL AVE,CHICAGO,IL,60638.0,2019-02-26T00:00:00,Pass w/ Conditions,41.777201,-87.761542


In [3]:
target_type = "restaurant"
chicago_coords = "41.881832,-87.623177"
radius = 10000

# rewrite params dict
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "location": chicago_coords,
    "types": target_type,
    "radius": radius,
    "key": gkey
}

print("Restaurants in Chicago, IL")

# Run request
response = requests.get(base_url, params)

chicago_restaurant = response.json()

# Print the JSON (pretty printed)
# print(json.dumps(chicago_restaurant, indent=4, sort_keys=True))

restaurants = []
chicago_rest_df = pd.DataFrame()

for index,restaurant in islice(chicago_df.iterrows(), 0, 200):    
    restr_name = restaurant['DBA Name']
    
    restr_inspection_id = restaurant['Inspection ID']

    # add keyword to params dict
    params['keyword'] = restr_name

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {restr_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        chicago_rest_df.loc[index, "Inspection ID"] = restr_inspection_id
        chicago_rest_df.loc[index, "Name"] = restr_name
        chicago_rest_df.loc[index, "Latitude"] = results[0]["geometry"]["location"]["lat"]
        chicago_rest_df.loc[index, "Longitude"] = results[0]["geometry"]["location"]["lng"]
        chicago_rest_df.loc[index, "Ratings"] =  results[0]["rating"]
        print(f"Record found for Restaurant : {restr_name}")
    except(KeyError, IndexError):
        print("Record not Found")
       

Restaurants in Chicago, IL
Retrieving Results for Index 0: PIZZERIA AROMA.
Record not Found
Retrieving Results for Index 1: PIZZERIA AROMA.
Record not Found
Retrieving Results for Index 2: CAFECITO.
Record found for Restaurant : CAFECITO
Retrieving Results for Index 3: JOHNSON'S REAL ICE CREAM.
Record not Found
Retrieving Results for Index 4: TONY'S KITCHEN #2 INC..
Record not Found
Retrieving Results for Index 5: EL SABOR POBLANO.
Record not Found
Retrieving Results for Index 6: BOOILL GALBI.
Record not Found
Retrieving Results for Index 7: THE GARAGE BAR AND SANDWICHES.
Record not Found
Retrieving Results for Index 8: RUKIYO'S.
Record found for Restaurant : RUKIYO'S
Retrieving Results for Index 9: HOANH LONG RESTAURANT, INC..
Record not Found
Retrieving Results for Index 10: EL REY DE LA COSTA.
Record found for Restaurant : EL REY DE LA COSTA
Retrieving Results for Index 11: LATIN PATIO.
Record not Found
Retrieving Results for Index 12: IYANZE.
Record found for Restaurant : IYANZE
Re

Record not Found
Retrieving Results for Index 93: LUZZAT RESTAURANT INC..
Record not Found
Retrieving Results for Index 94: ALEX LA MICHOACANA.
Record not Found
Retrieving Results for Index 95: JIANG'S KITCHEN INC..
Record not Found
Retrieving Results for Index 96: ANTOJITOS PUEBLITA INC.
Record found for Restaurant : ANTOJITOS PUEBLITA INC
Retrieving Results for Index 97: SUSHI WORLD.
Record not Found
Retrieving Results for Index 98: PEPES RESTAURANT.
Record found for Restaurant : PEPES RESTAURANT
Retrieving Results for Index 99: ANNA PURANA FAST FOOD.
Record not Found
Retrieving Results for Index 100: GREAT SEA CHINESE RESTAURANT.
Record found for Restaurant : GREAT SEA CHINESE RESTAURANT
Retrieving Results for Index 101: COLUTA'S PIZZA.
Record not Found
Retrieving Results for Index 102: GREAT SEA CHINESE RESTAURANT.
Record found for Restaurant : GREAT SEA CHINESE RESTAURANT
Retrieving Results for Index 103: Tortuga's Cantina.
Record found for Restaurant : Tortuga's Cantina
Retrievin

Record found for Restaurant : MAKO
Retrieving Results for Index 184: POZOLERIA IGUALA IN CHICAGO INC..
Record found for Restaurant : POZOLERIA IGUALA IN CHICAGO INC.
Retrieving Results for Index 185: LOLAS CONEY LLC.
Record not Found
Retrieving Results for Index 186: SUBWAY SANDWICHES AND SALAD.
Record not Found
Retrieving Results for Index 187: FOX'S BEVERLY PUB.
Record not Found
Retrieving Results for Index 188: POPEYES.
Record found for Restaurant : POPEYES
Retrieving Results for Index 189: SUBWAY.
Record found for Restaurant : SUBWAY
Retrieving Results for Index 190: BABY BURRITOS.
Record not Found
Retrieving Results for Index 191: GORDO'S TAMALES.
Record found for Restaurant : GORDO'S TAMALES
Retrieving Results for Index 192: MARWA GRILL.
Record not Found
Retrieving Results for Index 193: GORDO TAMALES.
Record found for Restaurant : GORDO TAMALES
Retrieving Results for Index 194: CAPT'N NEMO.
Record found for Restaurant : CAPT'N NEMO
Retrieving Results for Index 195: THE DOG HOUSE

In [13]:
chicago_rest_df.dropna()

,Inspection ID,Name,Latitude,Longitude,Ratings
2,2268527.0,CAFECITO,41.898192,-87.621275,4.7
8,2268512.0,RUKIYO'S,41.900051,-87.637045,0.0
10,2268510.0,EL REY DE LA COSTA,41.879322,-87.668978,4.3
12,2262183.0,IYANZE,41.966191,-87.657573,3.9
13,2268495.0,GIORDANO'S RESTAURANT,41.885130,-87.623761,4.4
14,2268487.0,LITTLE KITCHEN,41.956707,-87.655293,3.9
15,2268477.0,YVOLINA'S TAMALES+GROCERIES,41.858237,-87.647300,4.5
18,2268469.0,ROSATI'S PIZZA,41.864247,-87.646698,3.3
20,2268456.0,DUNKIN DONUTS,41.886154,-87.631501,4.0
21,2268457.0,BURRITO STATION,41.799258,-87.591943,4.0


In [12]:
chicago_rest_df.to_csv('updated_ratings_table.csv')

In [14]:
chicago_rest_df.to_csv('ratings.csv')